# Times obtained during acquisition test (5 Orgs - 1 Peer Node per Org)

### Imports

In [1]:
import sys
sys.path.insert(1, '..\code')
from acquisitor import *
import threading
import requests
import json
import random

### Global variables

In [2]:
API_ENDPOINT = "http://35.241.200.124:3000/api/"
NS = "ertis.uma.nuclear"

### Function

In [3]:
def addAcquisitionTest(num_acquisitors, num_tubes):
    acquisitor_threads = []
    acquisitions_per_worker = int(num_tubes/num_acquisitors)

    # Check for next acquisition id
    resource_url = f"{API_ENDPOINT}{NS}.Acquisition"
    r = requests.get(resource_url)
    next_id = len(r.json()) + 1

    # Create threads
    for i in range(num_acquisitors):
        thread = Acquisitor(i, f"Acquisitor-{i}", acquisitions_per_worker, next_id+acquisitions_per_worker*i, API_ENDPOINT, NS)
        acquisitor_threads.append(thread)

    # Start threads
    for i in range(num_acquisitors):
        acquisitor_threads[i].start()

    # Join threads
    for i in range(num_acquisitors):
        acquisitor_threads[i].join()
        
    # Print results
    min = acquisitor_threads[0].min
    min5avg = acquisitor_threads[0].min5avg
    avg = acquisitor_threads[0].avg
    max = acquisitor_threads[0].max
    max5avg = acquisitor_threads[0].max5avg
    acquisitor_threads[0].printResults()
    for i in range(1, num_acquisitors):
        if acquisitor_threads[i].min < min:
            min = acquisitor_threads[i].min
            
        min5avg = min5avg + acquisitor_threads[i].min5avg

        avg = avg + acquisitor_threads[i].avg

        if acquisitor_threads[i].max > max:
            max = acquisitor_threads[i].max
            
        max5avg = max5avg + acquisitor_threads[i].max5avg

        acquisitor_threads[i].printResults()

    print(f"MIN --> {min}")
    print(f"MIN 5 AVG --> {min5avg/num_acquisitors}")
    print(f"AVG --> {avg/num_acquisitors}")
    print(f"MAX --> {max}")
    print(f"MAX 5 AVG --> {max5avg/num_acquisitors}")

### Execution of the test with one single acquisitor (acquisition tx also triggers the execution of an analysis)

In [ ]:
addAcquisitionTest(1, 100) #Acquisitors, Acquisitions to do